In [1]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            tender_amount = tender.get('value', {}).get('amount')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'amount': tender_amount,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2015_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [26]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2016_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [27]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2017_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [28]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2018_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [29]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2019_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [30]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2020_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [31]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2021_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [32]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2022_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [33]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2023_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [34]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2024_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [35]:
import json
import csv
import os
from datetime import datetime
import re

def normalize_data_to_csv(json_file_path, output_folder):
    # Extraer el año del nombre del archivo JSON
    year_match = re.search(r'_(\d{4})_', json_file_path)
    year = year_match.group(1) if year_match else ""
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    try:
        # Cargar el archivo JSON
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        return
    
    # Inicializar diccionarios para almacenar los datos de cada tabla
    tables = {
        'releases': [],
        'planning': [],
        'budget': [],
        'tender': [],
        'tender_period': [],
        'enquiry_period': [],
        'award_period': [],
        'lot': [],
        'classification': [],
        'unit': [],
        'item': [],
        'additional_classification': [],
        'enquiry': [],
        'enquiry_author': [],
        'award': [],
        'award_supplier': [],
        'award_item': [],
        'contract': [],
        'party': [],
        'party_identifier': [],
        'auction': [],
        'auction_round': [],
        'bid': [],
        'bid_tenderer': [],
        'winning_bid': []
    }
    
    for record in data:
        # Extraer el release (primer elemento del array releases)
        releases = record.get('releases', [])
        if not releases:
            continue  # Saltar si no hay releases
            
        release = releases[0]
        
        # Extraer OCID del release (no del record principal)
        ocid = release.get('ocid')
        if not ocid:
            print(f"Advertencia: Release sin OCID: {release.get('id', 'ID no disponible')}")
            continue
            
        release_id = release.get('id')
        date = release.get('date')
        tags = ','.join(release.get('tag', []))
        buyer_id = release.get('buyer', {}).get('id')
        buyer_name = release.get('buyer', {}).get('name')
        initiation_type = release.get('initiationType')
        language = release.get('language')
        publisher_uid = record.get('publisher', {}).get('uid')
        published_date = record.get('publishedDate')
        
        # Tabla releases
        tables['releases'].append({
            'ocid': ocid,
            'release_id': release_id,
            'date': date,
            'tag': tags,
            'buyer_id': buyer_id,
            'buyer_name': buyer_name,
            'initiationType': initiation_type,
            'language': language,
            'publisher_uid': publisher_uid,
            'publishedDate': published_date
        })
        
        # Procesar planning
        planning = release.get('planning', {})
        budget_id = planning.get('budget', {}).get('id')
        rationale = planning.get('rationale')
        
        if budget_id or rationale:
            tables['planning'].append({
                'ocid': ocid,
                'budget_id': budget_id,
                'rationale': rationale
            })
        
        # Procesar budget si existe
        if budget_id:
            budget = planning.get('budget', {})
            budget_amount = budget.get('amount')
            budget_currency = budget.get('currency')
            
            if budget_amount or budget_currency:
                tables['budget'].append({
                    'ocid': ocid,
                    'budget_id': budget_id,
                    'amount': budget_amount,
                    'currency': budget_currency
                })
        
        # Procesar tender
        tender = release.get('tender', {})
        if tender:
            tender_id = tender.get('id')
            title = tender.get('title')
            description = tender.get('description')
            status = tender.get('status')
            procuring_entity_id = tender.get('procuringEntity', {}).get('id')
            procuring_entity_name = tender.get('procuringEntity', {}).get('name')
            procurement_method = tender.get('procurementMethod')
            main_procurement_category = tender.get('mainProcurementCategory')
            procurement_method_details = tender.get('procurementMethodDetails')
            number_of_tenderers = tender.get('numberOfTenderers')
            has_enquiries = 1 if tender.get('hasEnquiries') else 0
            award_criteria = tender.get('awardCriteria')
            
            tables['tender'].append({
                'ocid': ocid,
                'tender_id': tender_id,
                'title': title,
                'description': description,
                'status': status,
                'procuringEntity_id': procuring_entity_id,
                'procuringEntity_name': procuring_entity_name,
                'procurementMethod': procurement_method,
                'mainProcurementCategory': main_procurement_category,
                'procurementMethodDetails': procurement_method_details,
                'numberOfTenderers': number_of_tenderers,
                'hasEnquiries': has_enquiries,
                'awardCriteria': award_criteria
            })
            
            # Procesar periodos
            tender_period = tender.get('tenderPeriod', {})
            if tender_period:
                tables['tender_period'].append({
                    'ocid': ocid,
                    'startDate': tender_period.get('startDate'),
                    'endDate': tender_period.get('endDate')
                })
            
            enquiry_period = tender.get('enquiryPeriod', {})
            if enquiry_period:
                tables['enquiry_period'].append({
                    'ocid': ocid,
                    'startDate': enquiry_period.get('startDate'),
                    'endDate': enquiry_period.get('endDate'),
                    'durationInDays': enquiry_period.get('durationInDays'),
                    'maxExtentDate': enquiry_period.get('maxExtentDate')
                })
            
            award_period = tender.get('awardPeriod', {})
            if award_period:
                tables['award_period'].append({
                    'ocid': ocid,
                    'startDate': award_period.get('startDate'),
                    'endDate': award_period.get('endDate'),
                    'durationInDays': award_period.get('durationInDays'),
                    'maxExtentDate': award_period.get('maxExtentDate')
                })
            
            # Procesar lots
            lots = tender.get('lots', [])
            for lot in lots:
                lot_id = lot.get('id')
                lot_title = lot.get('title')
                lot_amount = lot.get('value', {}).get('amount')
                lot_currency = lot.get('value', {}).get('currency')
                has_electronic_auction = 1 if lot.get('techniques', {}).get('hasElectronicAuction') else 0
                
                tables['lot'].append({
                    'lot_id': lot_id,
                    'ocid': ocid,
                    'title': lot_title,
                    'amount': lot_amount,
                    'currency': lot_currency,
                    'hasElectronicAuction': has_electronic_auction
                })
            
            # Procesar items
            items = tender.get('items', [])
            for item in items:
                item_id = item.get('id')
                lot_id = item.get('relatedLot')
                description = item.get('description')
                quantity = item.get('quantity')
                unit_value = item.get('unit', {}).get('value', {})
                amount = unit_value.get('amount')
                currency = unit_value.get('currency')
                
                # Procesar classification
                classification = item.get('classification', {})
                if classification:
                    classification_id = classification.get('id')
                    scheme = classification.get('scheme')
                    class_description = classification.get('description')
                    
                    tables['classification'].append({
                        'ocid': ocid,
                        'classification_id': classification_id,
                        'scheme': scheme,
                        'description': class_description
                    })
                
                # Procesar unit
                unit = item.get('unit', {})
                if unit and isinstance(unit, dict):
                    unit_id = unit.get('id')
                    unit_name = unit.get('name')
                    unit_scheme = unit.get('scheme')
                    
                    tables['unit'].append({
                        'ocid': ocid,
                        'unit_id': unit_id,
                        'name': unit_name,
                        'scheme': unit_scheme
                    })
                
                tables['item'].append({
                    'item_id': item_id,
                    'ocid': ocid,
                    'lot_id': lot_id,
                    'description': description,
                    'quantity': quantity,
                    'amount': amount,
                    'currency': currency,
                    'classification_id': classification.get('id'),
                    'unit_id': unit.get('id') if isinstance(unit, dict) else None
                })
                
                # Procesar additionalClassifications
                additional_classifications = item.get('additionalClassifications', [])
                for ac in additional_classifications:
                    ac_id = ac.get('id')
                    scheme = ac.get('scheme')
                    description = ac.get('description')
                    uri = ac.get('uri')
                    
                    tables['additional_classification'].append({
                        'ocid': ocid,
                        'id': ac_id,
                        'item_id': item_id,
                        'scheme': scheme,
                        'description': description,
                        'uri': uri
                    })
            
            # Procesar enquiries
            enquiries = tender.get('enquiries', [])
            for enquiry in enquiries:
                enquiry_id = enquiry.get('id')
                enquiry_date = enquiry.get('date')
                description = enquiry.get('description')
                answer = enquiry.get('answer')
                date_answered = enquiry.get('dateAnswered')
                
                tables['enquiry'].append({
                    'enquiry_id': enquiry_id,
                    'ocid': ocid,
                    'date': enquiry_date,
                    'description': description,
                    'answer': answer,
                    'dateAnswered': date_answered
                })
                
                # Procesar author del enquiry
                author = enquiry.get('author', {})
                if author:
                    author_id = author.get('id')
                    author_name = author.get('name')
                    
                    # Primero insertar el autor como party si no existe
                    if not any(p['party_id'] == author_id for p in tables['party']):
                        tables['party'].append({
                            'party_id': author_id,
                            'ocid': ocid,
                            'name': author_name,
                            'role': 'enquiry_author',
                            'streetAddress': None,
                            'locality': None,
                            'region': None,
                            'countryName': None,
                            'postalCode': None
                        })
                    
                    # Luego la relación entre enquiry y author
                    tables['enquiry_author'].append({
                        'ocid': ocid,
                        'enquiry_id': enquiry_id,
                        'author_id': author_id
                    })
        
        # Procesar awards
        awards = release.get('awards', [])
        for award in awards:
            award_id = award.get('id')
            award_date = award.get('date')
            award_description = award.get('description')
            award_status = award.get('status')
            award_value = award.get('value', {})
            award_amount = award_value.get('amount')
            award_currency = award_value.get('currency')
            
            tables['award'].append({
                'award_id': award_id,
                'ocid': ocid,
                'date': award_date,
                'description': award_description,
                'status': award_status,
                'amount': award_amount,
                'currency': award_currency
            })
            
            # Procesar suppliers del award
            suppliers = award.get('suppliers', [])
            for supplier in suppliers:
                supplier_id = supplier.get('id')
                supplier_name = supplier.get('name')
                
                # Insertar supplier como party si no existe
                if not any(p['party_id'] == supplier_id for p in tables['party']):
                    tables['party'].append({
                        'party_id': supplier_id,
                        'ocid': ocid,
                        'name': supplier_name,
                        'role': 'supplier',
                        'streetAddress': None,
                        'locality': None,
                        'region': None,
                        'countryName': None,
                        'postalCode': None
                    })
                
                # Relación award-supplier
                tables['award_supplier'].append({
                    'ocid': ocid,
                    'award_id': award_id,
                    'supplier_id': supplier_id
                })
            
            # Procesar items del award
            award_items = award.get('items', [])
            for item in award_items:
                item_id = item.get('id')
                
                # Verificar si el item existe en la tabla items
                if any(i['item_id'] == item_id for i in tables['item']):
                    tables['award_item'].append({
                        'ocid': ocid,
                        'award_id': award_id,
                        'item_id': item_id
                    })
        
        # Procesar contracts
        contracts = release.get('contracts', [])
        for contract in contracts:
            contract_id = contract.get('id')
            award_id = contract.get('awardID')
            contract_status = contract.get('status')
            contract_value = contract.get('value', {})
            contract_amount = contract_value.get('amount')
            contract_currency = contract_value.get('currency')
            date_signed = contract.get('dateSigned')
            contract_period = contract.get('period', {})
            duration_in_days = contract_period.get('durationInDays')
            start_date = contract_period.get('startDate')
            end_date = contract_period.get('endDate')
            
            tables['contract'].append({
                'contract_id': contract_id,
                'ocid': ocid,
                'award_id': award_id,
                'status': contract_status,
                'amount': contract_amount,
                'currency': contract_currency,
                'dateSigned': date_signed,
                'durationInDays': duration_in_days,
                'startDate': start_date,
                'endDate': end_date
            })
        
        # Procesar parties
        parties = release.get('parties', [])
        for party_data in parties:
            party_id = party_data.get('id')
            party_name = party_data.get('name')
            roles = ','.join(party_data.get('roles', []))
            address = party_data.get('address', {})
            
            # Verificar si la party ya existe para actualizarla o crearla
            existing_party = next((p for p in tables['party'] if p['party_id'] == party_id), None)
            if existing_party:
                # Actualizar roles si es necesario
                existing_party['role'] = roles
                existing_party.update({
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            else:
                tables['party'].append({
                    'party_id': party_id,
                    'ocid': ocid,
                    'name': party_name,
                    'role': roles,
                    'streetAddress': address.get('streetAddress'),
                    'locality': address.get('locality'),
                    'region': address.get('region'),
                    'countryName': address.get('countryName'),
                    'postalCode': address.get('postalCode')
                })
            
            # Procesar identifier de party
            identifier = party_data.get('identifier', {})
            if identifier:
                tables['party_identifier'].append({
                    'ocid': ocid,
                    'party_id': party_id,
                    'scheme': identifier.get('scheme'),
                    'legalName': identifier.get('legalName')
                })
        
        # Procesar auctions
        auctions = release.get('auctions', [])
        for auction in auctions:
            auction_id = auction.get('id')
            auction_status = auction.get('status')
            auction_period = auction.get('period', {})
            auction_start = auction_period.get('startDate')
            auction_end = auction_period.get('endDate')
            
            tables['auction'].append({
                'auction_id': auction_id,
                'ocid': ocid,
                'status': auction_status,
                'startDate': auction_start,
                'endDate': auction_end
            })
            
            # Procesar stages (rondas) de la subasta
            stages = auction.get('stages', [])
            for stage in stages:
                stage_id = stage.get('id')
                stage_title = stage.get('title')
                
                # Procesar rounds de la stage
                rounds = stage.get('rounds', [])
                for round_data in rounds:
                    round_id = round_data.get('id')
                    round_period = round_data.get('period', {})
                    round_start = round_period.get('startDate')
                    round_end = round_period.get('endDate')
                    
                    tables['auction_round'].append({
                        'ocid': ocid,
                        'round_id': round_id,
                        'auction_id': auction_id,
                        'startDate': round_start,
                        'endDate': round_end
                    })
                    
                    # Procesar bids de la stage
                    bids = stage.get('bids', [])
                    for bid in bids:
                        bid_id = bid.get('id')
                        bid_date = bid.get('date')
                        bid_value = bid.get('value', {})
                        bid_amount = bid_value.get('amount')
                        bid_currency = bid_value.get('currency')
                        bid_status = 'winning' if bid in auction.get('winningBids', []) else 'regular'
                        
                        tables['bid'].append({
                            'ocid': ocid,
                            'bid_id': bid_id,
                            'round_id': round_id,
                            'date': bid_date,
                            'amount': bid_amount,
                            'currency': bid_currency,
                            'status': bid_status
                        })
                        
                        # Procesar tenderers de la bid
                        tenderers = bid.get('tenderers', [])
                        for tenderer in tenderers:
                            tenderer_id = tenderer.get('id')
                            tenderer_name = tenderer.get('name')
                            
                            # Insertar tenderer como party si no existe
                            if not any(p['party_id'] == tenderer_id for p in tables['party']):
                                tables['party'].append({
                                    'party_id': tenderer_id,
                                    'ocid': ocid,
                                    'name': tenderer_name,
                                    'role': 'tenderer',
                                    'streetAddress': None,
                                    'locality': None,
                                    'region': None,
                                    'countryName': None,
                                    'postalCode': None
                                })
                            
                            # Relación bid-tenderer
                            tables['bid_tenderer'].append({
                                'ocid': ocid,
                                'bid_id': bid_id,
                                'tenderer_id': tenderer_id
                            })
            
            # Procesar winningBids
            winning_bids = auction.get('winningBids', [])
            for winning_bid in winning_bids:
                winning_bid_id = winning_bid.get('id')
                
                tables['winning_bid'].append({
                    'ocid': ocid,
                    'bid_id': winning_bid_id,
                    'auction_id': auction_id
                })
    
    # Escribir todos los datos a archivos CSV
    for table_name, rows in tables.items():
        if rows:
            # Modificamos esta línea para incluir el año
            csv_file_path = os.path.join(output_folder, f"{table_name}_{year}.csv")
            
            # Obtener los nombres de las columnas del primer registro
            fieldnames = rows[0].keys() if rows else []
            
            with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
    
    print(f"Datos normalizados y exportados a archivos CSV en la carpeta: {output_folder}")

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar el archivo JSON y exportar a CSV
    json_file_path = "releases_2025_subasta_inversa_electronica.json"
    output_folder = "compras_publicas_csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(json_file_path):
        print(f"Error: El archivo {json_file_path} no existe")
    else:
        normalize_data_to_csv(json_file_path, output_folder)

Datos normalizados y exportados a archivos CSV en la carpeta: compras_publicas_csv


In [36]:
import pandas as pd
import os

# Directorio donde están los archivos CSV
directorio = "C:/Users/AndrésL/Documents/OCDS2025/compras_publicas_csv"

# Obtener todos los archivos CSV en el directorio
archivos = os.listdir(directorio)
archivos_csv = [f for f in archivos if f.endswith('.csv')]

# Crear un diccionario para agrupar archivos por tipo (prefijo)
grupos = {}
for archivo in archivos_csv:
    # Extraer el prefijo (nombre sin año y extensión)
    partes = archivo.split('_')
    if len(partes) > 1 and partes[-1].replace('.csv', '').isdigit():
        prefijo = '_'.join(partes[:-1])
        if prefijo not in grupos:
            grupos[prefijo] = []
        grupos[prefijo].append(archivo)

# Procesar cada grupo
for prefijo, lista_archivos in grupos.items():
    print(f"Procesando {prefijo}...")
    
    # Leer y concatenar todos los archivos del grupo
    dfs = []
    for archivo in lista_archivos:
        ruta_completa = os.path.join(directorio, archivo)
        df = pd.read_csv(ruta_completa)
        dfs.append(df)
    
    # Concatenar todos los DataFrames
    df_final = pd.concat(dfs, ignore_index=True)
    
    # Guardar el resultado
    nombre_salida = f"{prefijo}_completo.csv"
    ruta_salida = os.path.join("C:/Users/AndrésL/Documents/OCDS2025/subastas_inversas", nombre_salida)
    df_final.to_csv(ruta_salida, index=False)
    print(f"Guardado: {nombre_salida}")

print("Proceso completado.")

Procesando additional_classification...
Guardado: additional_classification_completo.csv
Procesando auction...
Guardado: auction_completo.csv
Procesando auction_round...
Guardado: auction_round_completo.csv
Procesando award...
Guardado: award_completo.csv
Procesando award_item...
Guardado: award_item_completo.csv
Procesando award_period...
Guardado: award_period_completo.csv
Procesando award_supplier...
Guardado: award_supplier_completo.csv
Procesando bid...
Guardado: bid_completo.csv
Procesando bid_tenderer...
Guardado: bid_tenderer_completo.csv
Procesando classification...
Guardado: classification_completo.csv
Procesando contract...
Guardado: contract_completo.csv
Procesando enquiry...
Guardado: enquiry_completo.csv
Procesando enquiry_author...
Guardado: enquiry_author_completo.csv
Procesando enquiry_period...
Guardado: enquiry_period_completo.csv
Procesando item...
Guardado: item_completo.csv
Procesando lot...
Guardado: lot_completo.csv
Procesando party...
Guardado: party_completo.c

In [21]:
import plotly.graph_objects as go

# Definimos los nodos (etiquetas)
labels = ["Entrada", "Proceso A", "Proceso B", "Salida 1", "Salida 2"]

# Definimos las conexiones entre nodos (índices)
source = [0, 0, 1, 1, 2]  # De qué nodo sale
target = [1, 2, 3, 4, 4]  # A qué nodo entra
value =  [8, 4, 2, 6, 4]  # Cantidad de flujo

# Crear el diagrama Sankey
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=10,
        line=dict(color="black", width=0.25),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
)])

fig.update_layout(title_text="Ejemplo de Diagrama Sankey", font_size=12)
fig.show()
